# Music Generation Using Deep Learning

## Data Source:
1. http://abc.sourceforge.net/NMD/

In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [2]:
data_directory = "Data/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = 'Data/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [3]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 
                #'1' is added because the correct label will be the next character in the sequence. 
                #Next character will be denoted by all_chars[batch_index * batch_chars + start + i + 1]
        yield X, Y

In [4]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(128, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [5]:
def training_model(data, epochs = 80):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("Data/log.csv", index = False)

In [6]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 05:21:09


In [7]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, 64, 512)             44544     
_________________________________________________________________
lstm (LSTM)                  (16, 64, 256)             787456    
_________________________________________________________________
dropout (Dropout)            (16, 64, 256)             0         
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 128)             197120    
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 128)             0         
_________________________________________________________________
time_distributed (TimeDistri (16, 64, 87)              11223     
_____________________________________________________________

Batch: 117, Loss: 3.0521674156188965, Accuracy: 0.2158203125
Batch: 118, Loss: 3.0683531761169434, Accuracy: 0.212890625
Batch: 119, Loss: 3.0476722717285156, Accuracy: 0.212890625
Batch: 120, Loss: 2.808389663696289, Accuracy: 0.27734375
Batch: 121, Loss: 2.8277523517608643, Accuracy: 0.2626953125
Batch: 122, Loss: 3.053068161010742, Accuracy: 0.197265625
Batch: 123, Loss: 2.992438793182373, Accuracy: 0.205078125
Batch: 124, Loss: 2.96481990814209, Accuracy: 0.212890625
Batch: 125, Loss: 2.8139872550964355, Accuracy: 0.244140625
Batch: 126, Loss: 2.794760227203369, Accuracy: 0.244140625
Batch: 127, Loss: 2.9547107219696045, Accuracy: 0.2177734375
Batch: 128, Loss: 2.898963451385498, Accuracy: 0.244140625
Batch: 129, Loss: 2.8356130123138428, Accuracy: 0.259765625
Batch: 130, Loss: 2.8548030853271484, Accuracy: 0.2509765625
Batch: 131, Loss: 2.8892736434936523, Accuracy: 0.2490234375
Batch: 132, Loss: 2.9532017707824707, Accuracy: 0.224609375
Batch: 133, Loss: 2.8672988414764404, Accur

Batch: 106, Loss: 2.0842814445495605, Accuracy: 0.431640625
Batch: 107, Loss: 2.1778621673583984, Accuracy: 0.4150390625
Batch: 108, Loss: 2.2024283409118652, Accuracy: 0.41796875
Batch: 109, Loss: 2.170295000076294, Accuracy: 0.431640625
Batch: 110, Loss: 1.9486485719680786, Accuracy: 0.4599609375
Batch: 111, Loss: 1.959223985671997, Accuracy: 0.44140625
Batch: 112, Loss: 2.1219582557678223, Accuracy: 0.4365234375
Batch: 113, Loss: 2.1310410499572754, Accuracy: 0.447265625
Batch: 114, Loss: 2.0614187717437744, Accuracy: 0.4306640625
Batch: 115, Loss: 2.125764846801758, Accuracy: 0.447265625
Batch: 116, Loss: 2.0832009315490723, Accuracy: 0.4189453125
Batch: 117, Loss: 2.1673216819763184, Accuracy: 0.4375
Batch: 118, Loss: 2.040391445159912, Accuracy: 0.4609375
Batch: 119, Loss: 2.072383165359497, Accuracy: 0.4560546875
Batch: 120, Loss: 2.0275182723999023, Accuracy: 0.43359375
Batch: 121, Loss: 2.048119068145752, Accuracy: 0.4560546875
Batch: 122, Loss: 2.143042802810669, Accuracy: 0.

Batch: 95, Loss: 1.6446454524993896, Accuracy: 0.498046875
Batch: 96, Loss: 1.7174867391586304, Accuracy: 0.525390625
Batch: 97, Loss: 1.6149303913116455, Accuracy: 0.544921875
Batch: 98, Loss: 1.6064021587371826, Accuracy: 0.5576171875
Batch: 99, Loss: 1.5786323547363281, Accuracy: 0.5283203125
Batch: 100, Loss: 1.6143004894256592, Accuracy: 0.517578125
Batch: 101, Loss: 1.6579291820526123, Accuracy: 0.5009765625
Batch: 102, Loss: 1.5734580755233765, Accuracy: 0.521484375
Batch: 103, Loss: 1.7303394079208374, Accuracy: 0.5400390625
Batch: 104, Loss: 1.5964443683624268, Accuracy: 0.5400390625
Batch: 105, Loss: 1.6817693710327148, Accuracy: 0.5107421875
Batch: 106, Loss: 1.7536201477050781, Accuracy: 0.4951171875
Batch: 107, Loss: 1.8400992155075073, Accuracy: 0.4873046875
Batch: 108, Loss: 1.7983027696609497, Accuracy: 0.50390625
Batch: 109, Loss: 1.8181300163269043, Accuracy: 0.4853515625
Batch: 110, Loss: 1.5419890880584717, Accuracy: 0.5478515625
Batch: 111, Loss: 1.6564356088638306

Batch: 70, Loss: 1.5186889171600342, Accuracy: 0.548828125
Batch: 71, Loss: 1.487682580947876, Accuracy: 0.54296875
Batch: 72, Loss: 1.3630952835083008, Accuracy: 0.576171875
Batch: 73, Loss: 1.4638984203338623, Accuracy: 0.5654296875
Batch: 74, Loss: 1.4021210670471191, Accuracy: 0.57421875
Batch: 75, Loss: 1.3344616889953613, Accuracy: 0.583984375
Batch: 76, Loss: 1.4699368476867676, Accuracy: 0.5126953125
Batch: 77, Loss: 1.488111138343811, Accuracy: 0.5185546875
Batch: 78, Loss: 1.4648025035858154, Accuracy: 0.5751953125
Batch: 79, Loss: 1.3166038990020752, Accuracy: 0.6318359375
Batch: 80, Loss: 1.3381892442703247, Accuracy: 0.564453125
Batch: 81, Loss: 1.4831500053405762, Accuracy: 0.51953125
Batch: 82, Loss: 1.451777458190918, Accuracy: 0.544921875
Batch: 83, Loss: 1.3210501670837402, Accuracy: 0.603515625
Batch: 84, Loss: 1.3771791458129883, Accuracy: 0.6005859375
Batch: 85, Loss: 1.358204960823059, Accuracy: 0.5869140625
Batch: 86, Loss: 1.5823769569396973, Accuracy: 0.5068359

...

Batch: 134, Loss: 0.4836463928222656, Accuracy: 0.83984375
Batch: 135, Loss: 0.4495064616203308, Accuracy: 0.8447265625
Batch: 136, Loss: 0.4914925694465637, Accuracy: 0.83984375
Batch: 137, Loss: 0.5365434885025024, Accuracy: 0.806640625
Batch: 138, Loss: 0.45196297764778137, Accuracy: 0.8486328125
Batch: 139, Loss: 0.4674414396286011, Accuracy: 0.8515625
Batch: 140, Loss: 0.4548678994178772, Accuracy: 0.849609375
Batch: 141, Loss: 0.5191956162452698, Accuracy: 0.8203125
Batch: 142, Loss: 0.5012630224227905, Accuracy: 0.826171875
Batch: 143, Loss: 0.49684208631515503, Accuracy: 0.833984375
Batch: 144, Loss: 0.5054967999458313, Accuracy: 0.8408203125
Batch: 145, Loss: 0.43347957730293274, Accuracy: 0.8505859375
Batch: 146, Loss: 0.4870107173919678, Accuracy: 0.8408203125
Batch: 147, Loss: 0.44165128469467163, Accuracy: 0.845703125
Batch: 148, Loss: 0.5076867341995239, Accuracy: 0.83984375
Batch: 149, Loss: 0.4945043623447418, Accuracy: 0.8525390625
Batch: 150, Loss: 0.5336127877235413,

Batch: 121, Loss: 0.48302704095840454, Accuracy: 0.8408203125
Batch: 122, Loss: 0.47177261114120483, Accuracy: 0.8486328125
Batch: 123, Loss: 0.4231867790222168, Accuracy: 0.8642578125
Batch: 124, Loss: 0.4660494327545166, Accuracy: 0.8447265625
Batch: 125, Loss: 0.49586236476898193, Accuracy: 0.833984375
Batch: 126, Loss: 0.49265730381011963, Accuracy: 0.8330078125
Batch: 127, Loss: 0.4006551206111908, Accuracy: 0.8671875
Batch: 128, Loss: 0.5593857765197754, Accuracy: 0.8310546875
Batch: 129, Loss: 0.4187878668308258, Accuracy: 0.8642578125
Batch: 130, Loss: 0.5368554592132568, Accuracy: 0.826171875
Batch: 131, Loss: 0.4716518223285675, Accuracy: 0.8427734375
Batch: 132, Loss: 0.5016157031059265, Accuracy: 0.837890625
Batch: 133, Loss: 0.4625709652900696, Accuracy: 0.8408203125
Batch: 134, Loss: 0.4848405122756958, Accuracy: 0.8271484375
Batch: 135, Loss: 0.46402591466903687, Accuracy: 0.841796875
Batch: 136, Loss: 0.5146574974060059, Accuracy: 0.841796875
Batch: 137, Loss: 0.4945314

Batch: 108, Loss: 0.49471595883369446, Accuracy: 0.8486328125
Batch: 109, Loss: 0.5203542113304138, Accuracy: 0.822265625
Batch: 110, Loss: 0.4548211097717285, Accuracy: 0.8515625
Batch: 111, Loss: 0.47661954164505005, Accuracy: 0.841796875
Batch: 112, Loss: 0.503929853439331, Accuracy: 0.83203125
Batch: 113, Loss: 0.5046536922454834, Accuracy: 0.833984375
Batch: 114, Loss: 0.5333083868026733, Accuracy: 0.8349609375
Batch: 115, Loss: 0.5334005355834961, Accuracy: 0.8330078125
Batch: 116, Loss: 0.47774797677993774, Accuracy: 0.8349609375
Batch: 117, Loss: 0.4654484689235687, Accuracy: 0.8388671875
Batch: 118, Loss: 0.41516637802124023, Accuracy: 0.853515625
Batch: 119, Loss: 0.3858949840068817, Accuracy: 0.8740234375
Batch: 120, Loss: 0.453195720911026, Accuracy: 0.8515625
Batch: 121, Loss: 0.45587965846061707, Accuracy: 0.833984375
Batch: 122, Loss: 0.4177314043045044, Accuracy: 0.865234375
Batch: 123, Loss: 0.4360010623931885, Accuracy: 0.85546875
Batch: 124, Loss: 0.4785129427909851,

Batch: 95, Loss: 0.4642770290374756, Accuracy: 0.8505859375
Batch: 96, Loss: 0.46168333292007446, Accuracy: 0.8603515625
Batch: 97, Loss: 0.38433554768562317, Accuracy: 0.8701171875
Batch: 98, Loss: 0.45265311002731323, Accuracy: 0.85546875
Batch: 99, Loss: 0.48439452052116394, Accuracy: 0.8408203125
Batch: 100, Loss: 0.4998970031738281, Accuracy: 0.82421875
Batch: 101, Loss: 0.4947868883609772, Accuracy: 0.8388671875
Batch: 102, Loss: 0.47045817971229553, Accuracy: 0.83984375
Batch: 103, Loss: 0.5108388662338257, Accuracy: 0.8349609375
Batch: 104, Loss: 0.44478458166122437, Accuracy: 0.85546875
Batch: 105, Loss: 0.48909157514572144, Accuracy: 0.8427734375
Batch: 106, Loss: 0.45923951268196106, Accuracy: 0.83984375
Batch: 107, Loss: 0.47758936882019043, Accuracy: 0.8505859375
Batch: 108, Loss: 0.45546209812164307, Accuracy: 0.85546875
Batch: 109, Loss: 0.4993124008178711, Accuracy: 0.830078125
Batch: 110, Loss: 0.4660986661911011, Accuracy: 0.8359375
Batch: 111, Loss: 0.462152659893035

Batch: 82, Loss: 0.48799455165863037, Accuracy: 0.8369140625
Batch: 83, Loss: 0.3807544708251953, Accuracy: 0.8662109375
Batch: 84, Loss: 0.4849298298358917, Accuracy: 0.8359375
Batch: 85, Loss: 0.4247489869594574, Accuracy: 0.86328125
Batch: 86, Loss: 0.5020750164985657, Accuracy: 0.8349609375
Batch: 87, Loss: 0.4436298608779907, Accuracy: 0.84765625
Batch: 88, Loss: 0.5081052780151367, Accuracy: 0.853515625
Batch: 89, Loss: 0.45188334584236145, Accuracy: 0.8447265625
Batch: 90, Loss: 0.4663424491882324, Accuracy: 0.8486328125
Batch: 91, Loss: 0.4406021237373352, Accuracy: 0.853515625
Batch: 92, Loss: 0.47631943225860596, Accuracy: 0.8525390625
Batch: 93, Loss: 0.4629727005958557, Accuracy: 0.83203125
Batch: 94, Loss: 0.49635109305381775, Accuracy: 0.8359375
Batch: 95, Loss: 0.476117879152298, Accuracy: 0.83203125
Batch: 96, Loss: 0.4843275249004364, Accuracy: 0.83984375
Batch: 97, Loss: 0.32754334807395935, Accuracy: 0.8818359375
Batch: 98, Loss: 0.46157747507095337, Accuracy: 0.8457

In [8]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 06:04:18


In [9]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

Epoch      Loss  Accuracy
0       1  2.774128  0.269531
1       2  1.942078  0.492188
2       3  1.594793  0.553711
3       4  1.454347  0.569336
4       5  1.349992  0.592773
..    ...       ...       ...
75     76  0.428267  0.855469
76     77  0.410729  0.858398
77     78  0.438185  0.852539
78     79  0.414675  0.874023
79     80  0.410107  0.869141

[80 rows x 3 columns]

In [10]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 06:04:18
